In [ ]:
%pip install langchain

Note: you may need to restart the kernel to use updated packages.


In [2]:
# Load a doc as text file
from langchain.document_loaders import TextLoader

loader = TextLoader("./data/history.md")
loader.load()

[Document(page_content='# A history lesson on Devops\n\n## Devopsdays\nDevopsdays is the conference that bootstrapped the word "Devops" by organising the first event.\nYou can visit the conference site via <https://devopsdays.org>\n\n## Devops Handbook\nThis book was written by Gene Kim\n\n## DevSecOps\nAs a reaction of Devops, Security got its own name and movement.\nNotably it talks about Shifting Security left.', metadata={'source': './data/history.md'})]

In [3]:
%pip install unstructured markdown


Note: you may need to restart the kernel to use updated packages.


In [5]:
from langchain.document_loaders import UnstructuredMarkdownLoader
loader = UnstructuredMarkdownLoader("./data/history.md")
loader.load()

[Document(page_content='A history lesson on Devops\n\nDevopsdays\n\nDevopsdays is the conference that bootstrapped the word "Devops" by organising the first event.\nYou can visit the conference site via https://devopsdays.org\n\nDevops Handbook\n\nThis book was written by Gene Kim\n\nDevSecOps\n\nAs a reaction of Devops, Security got its own name and movement.\nNotably it talks about Shifting Security left.', metadata={'source': './data/history.md'})]

In [6]:
from langchain.document_loaders import DirectoryLoader

loader = DirectoryLoader(
    "./",
    glob="data/**/*.*",
    use_multithreading=True,
    max_concurrency=4,
    show_progress=True,
)
loader.load()


100%|██████████| 1/1 [00:00<00:00, 72.22it/s]


[Document(page_content='A history lesson on Devops\n\nDevopsdays\n\nDevopsdays is the conference that bootstrapped the word "Devops" by organising the first event.\nYou can visit the conference site via https://devopsdays.org\n\nDevops Handbook\n\nThis book was written by Gene Kim\n\nDevSecOps\n\nAs a reaction of Devops, Security got its own name and movement.\nNotably it talks about Shifting Security left.', metadata={'source': 'data/history.md'})]

In [7]:
# https://python.langchain.com/docs/integrations/document_loaders/web_base
from langchain.document_loaders import WebBaseLoader
loader = WebBaseLoader("https://jedi.be")

data = loader.load()
print(data)


[Document(page_content='\n\n\n\n\n  Personal website of Patrick Debois – JEDI - Just Enough Documented Information Blog\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n            [Home]\n            [Blog]\n            [Contact]\n         -\n            [Talks]\n            [Bio]\n            [Customers]\n          \n\n\n\n\n\n\n\n\n\nSubscribe via email\n\n\n\n\n\nPersonal website of Patrick Debois\n\n\nA warm welcome to you !\n\nDuring 15 years of consultancy, I’ve assumed different roles within large enterprises ranging from developer, network specialist, system administrator, tester and project manager.  And because I’ve lived and experienced each role, I can talk to both manager, developer and IT people. Each in their own language.\n\nThis allows me to break past silo-based organizational boundaries resulting in smoother project delivery.\n\nI currently specialize in applying Agile techniques in infrastructure integration projects or what is sometimes called devops; agil